In [ ]:

import os
import numpy as np
from PIL import Image
import torch
from transformers import AutoImageProcessor, ViTModel
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy import stats

# --------------------------
# Configurações do dataset
# --------------------------
DIR_MELANOMA = r'data\\melanoma'
DIR_NAEVUS   = r'data\\naevus'
EXT          = ('.jpg', '.png', '.jpeg')  # ajuste se necessário

# --------------------------
# Configurações do modelo
# --------------------------
MODEL_NAME = "google/vit-base-patch16-224-in21k"
BATCH_SIZE = 8  # pode ajustar conforme sua GPU/CPU
SEED       = 42
NFOLDS     = 5  # K-Fold estratificado

torch.manual_seed(SEED)
np.random.seed(SEED)

# --------------------------
# Carregar lista de imagens + rótulos
# --------------------------
def list_images_from_dir(d, exts):
    files = sorted([os.path.join(d, f) for f in os.listdir(d) if f.lower().endswith(exts)])
    return files

paths_melanoma = list_images_from_dir(DIR_MELANOMA, EXT)
paths_naevus   = list_images_from_dir(DIR_NAEVUS, EXT)

X_paths = np.array(paths_melanoma + paths_naevus)
y       = np.array([1]*len(paths_melanoma) + [0]*len(paths_naevus))

print(f"# Imagens: {len(X_paths)} (melanoma={len(paths_melanoma)}, naevus={len(paths_naevus)})")

# --------------------------
# Carregar ViT e processor
# --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
model = ViTModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

# --------------------------
# Extrair last_hidden_state para todas as imagens
# Saída: numpy array shape (N, tokens, hidden_dim)
# --------------------------
def load_image_rgb(path):
    return Image.open(path).convert('RGB')

def extract_last_hidden_states(paths, batch_size=8):
    all_feats = []
    with torch.no_grad():
        for i in range(0, len(paths), batch_size):
            batch_paths = paths[i:i+batch_size]
            imgs = [load_image_rgb(p) for p in batch_paths]
            inputs = image_processor(imgs, return_tensors="pt").to(device)
            outputs = model(**inputs)
            # last_hidden_state: (B, tokens, hidden_dim)
            feats = outputs.last_hidden_state.detach().cpu().numpy()
            all_feats.append(feats)
    return np.concatenate(all_feats, axis=0)

print("Extraindo last_hidden_state...")
LH = extract_last_hidden_states(X_paths, batch_size=BATCH_SIZE)
# LH shape: (N, tokens, hidden_dim) -> tokens=197, hidden_dim=768 para ViT-Base
N, T, D = LH.shape
print(f"last_hidden_state shape = {LH.shape}")

# --------------------------
# Métodos de agregação
# Cada função retorna vetor (D,) por imagem
# Por padrão, excetuando 'cls', agregações usam APENAS patches (tokens[1:])
# --------------------------
def agg_cls(x):  # x: (tokens, D)
    return x[0]  # [CLS] token (posição 0)

def agg_mean(x):
    return x[1:].mean(axis=0)

def agg_median(x):
    return np.median(x[1:], axis=0)

def agg_max(x):
    return x[1:].max(axis=0)

def agg_min(x):
    return x[1:].min(axis=0)

def agg_l2_norm(x):
    # Norma L2 por dimensão (agregando sobre tokens): sqrt(sum_j x_j^2)
    return np.sqrt(np.sum(np.square(x[1:]), axis=0))

def agg_energy(x):
    # Energia por dimensão (sum_j x_j^2)
    return np.sum(np.square(x[1:]), axis=0)

def agg_gem(x, p=3.0):
    # GeM pooling por dimensão: (mean(x^p))^(1/p), tokens sem CLS
    xp = np.power(np.clip(x[1:], a_min=0.0, a_max=None), p)  # clip para evitar negativos com p fracionário
    return np.power(xp.mean(axis=0), 1.0/p)

def agg_logsumexp(x):
    # LogSumExp por dimensão: log(sum(exp(x)))
    # Estável numericamente
    a = x[1:]
    m = np.max(a, axis=0, keepdims=True)
    return (m + np.log(np.sum(np.exp(a - m), axis=0, keepdims=True))).ravel()

AGG_FUNCS = {
    "cls": agg_cls,
    "mean": agg_mean,
    "median": agg_median,
    "max": agg_max,
    "min": agg_min,
    "l2_norm": agg_l2_norm,
    "energy": agg_energy,
    "gem_p3": lambda x: agg_gem(x, p=3.0),
    "logsumexp": agg_logsumexp,
}

# --------------------------
# Pré-computar features agregadas (N, D) para cada método
# --------------------------
def apply_aggregation_all(LH, agg_func):
    return np.stack([agg_func(LH[i]) for i in range(LH.shape[0])], axis=0)

print("Agregando features por método...")
agg_features = {name: apply_aggregation_all(LH, fn) for name, fn in AGG_FUNCS.items()}
for k, v in agg_features.items():
    print(f"  {k}: {v.shape}")

# --------------------------
# K-Fold estratificado + LDA
# Métrica: AUC ROC (positiva=classe 1) + Accuracy
# --------------------------
def evaluate_lda_kfold(X, y, n_splits=5, seed=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    aucs, accs = [], []
    for tr_idx, te_idx in skf.split(X, y):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]

        # StandardScaler por fold (fit só no treino)
        scaler = StandardScaler()
        X_tr = scaler.fit_transform(X_tr)
        X_te = scaler.transform(X_te)

        lda = LinearDiscriminantAnalysis(solver='svd', shrinkage=None)
        lda.fit(X_tr, y_tr)
        y_proba = lda.predict_proba(X_te)[:, -1]
        y_pred  = lda.predict(X_te)

        aucs.append(roc_auc_score(y_te, y_proba))
        accs.append(accuracy_score(y_te, y_pred))
    return np.array(aucs), np.array(accs)

print("Avaliando LDA em K-Fold (estratificado)...")
results = {}
for name, X in agg_features.items():
    aucs, accs = evaluate_lda_kfold(X, y, n_splits=NFOLDS, seed=SEED)
    results[name] = {"auc": aucs, "acc": accs}
    print(f"  {name}: AUC média={aucs.mean():.4f} ± {aucs.std():.4f} | ACC média={accs.mean():.4f} ± {accs.std():.4f}")

# --------------------------
# Teste de Friedman (usando AUC por fold)
# --------------------------
# Organiza AUCs por método -> forma: (k métodos, N folds)
methods = list(results.keys())
k = len(methods)
auc_matrix = np.stack([results[m]["auc"] for m in methods], axis=0)  # (k, N)
N = auc_matrix.shape[1]

# Friedman (scipy.stats faz ranking internamente)
friedman_stat, friedman_p = stats.friedmanchisquare(*[auc_matrix[i] for i in range(k)])
print("\n# Teste de Friedman sobre AUCs (folds como blocos)")
print(f"  estatística={friedman_stat:.4f}, p-valor={friedman_p:.4f}")

# --------------------------
# Nemenyi (pós-hoc) com ranks médios
# CD = q_alpha * sqrt(k*(k+1)/(6*N)), q_alpha da distribuição studentized range
# Tenta usar scipy.stats.studentized_range; se não houver, usa tabela aproximada
# --------------------------
def mean_ranks(matrix):
    # matrix: (k, N) valores; rank por coluna (fold): melhor AUC -> rank 1
    ranks = np.zeros_like(matrix)
    for j in range(N):
        col = matrix[:, j]
        # rankdata coloca menor como 1; queremos maior (melhor) como 1 -> rank sobre -col
        r = stats.rankdata(-col, method='average')
        ranks[:, j] = r
    return ranks.mean(axis=1)  # (k,)

def q_alpha_nemenyi(k, alpha=0.05):
    try:
        # studentized_range.isf(alpha, k, np.inf) retorna q_alpha
        from scipy.stats import studentized_range
        return studentized_range.isf(alpha, k, np.inf)
    except Exception:
        # Tabela aproximada (Demsar 2006; ∞ df) para alpha=0.05
        table = {
            2: 1.960,  # não usual
            3: 2.343,
            4: 2.569,
            5: 2.728,
            6: 2.850,
            7: 2.948,
            8: 3.031,
            9: 3.102,
            10: 3.164,
        }
        return table.get(k, 3.164)  # fallback conservador
    # Nota: valores podem variar ligeiramente conforme tabela/implementação.

avg_ranks = mean_ranks(auc_matrix)  # (k,)
order = np.argsort(avg_ranks)       # menor rank (melhor) primeiro
sorted_methods = [methods[i] for i in order]
sorted_ranks   = avg_ranks[order]

q_alpha = q_alpha_nemenyi(k, alpha=0.05)
CD = q_alpha * np.sqrt(k*(k+1)/(6.0*N))

print("\n# Nemenyi pós-hoc (baseado em ranks médios de AUC)")
print(f"  k={k}, N(folds)={N}, q_alpha≈{q_alpha:.4f}, CD={CD:.4f}")
print("  Métodos ordenados por rank médio (menor=melhor):")
for m, r in zip(sorted_methods, sorted_ranks):
    print(f"    {m}: rank médio={r:.3f}")

# Pares significativamente diferentes se |rank_i - rank_j| > CD
print("\n  Pares com diferença de rank > CD (significativos a ~5%):")
sig_pairs = []
for i in range(k):
    for j in range(i+1, k):
        diff = abs(avg_ranks[i] - avg_ranks[j])
        if diff > CD:
            sig_pairs.append((methods[i], methods[j], diff))
            print(f"    {methods[i]} vs {methods[j]}: Δrank={diff:.3f} > CD")

if not sig_pairs:
    print("    Nenhum par significativo a ~5% pelo Nemenyi.")


# --------------------------
# Observações:
# - As agregações (exceto 'cls') consideram somente patches (tokens[1:]).
# - LDA é treinado por fold com StandardScaler aplicado (fit no treino, transform no teste).
# - Métrica principal usada para estatística é AUC; você pode trocar para ACC se preferir.
# - Caso tenha muitos dados, considere salvar/recuperar LH pré-computado para evitar reextração.
# --------------------------
